<span style="font-family: Arial; font-size: 25px; font-weight: bold;">
Import Packages
</span>

In [115]:
import pandas as pd
import numpy as np
!pip install pandas openpyxl
from dateutil.relativedelta import relativedelta

<span style="font-family: Arial; font-size: 25px; font-weight: bold;">
Loading the Dataset
</span>

In [116]:
df = pd.read_csv("HR payroll Dataset.csv") # replace with your actual file name

<span style="font-family: Arial; font-size: 25px; font-weight: bold;">
Remove Duplicate Employee IDs</span>

In [119]:
df = df.drop_duplicates(subset='EmployeeID', keep='first')

<span style="font-family: Arial; font-size: 25px; font-weight: bold;">
Convert Salary from Text to Numeric</span>

In [121]:
df['Salary'] = (
    df['Salary']
    .astype(str)
    .str.replace(r'[^0-9.]', '', regex=True)
    .replace('', np.nan)
    .astype(float)
)

<span style="font-family: Arial; font-size: 25px; font-weight: bold;">
Standardize Department Names</span>

In [123]:
department_mapping = {
    'Mktg': 'Marketing',
    'Marketing': 'Marketing',
    'Sale': 'Sales',
    'Sales': 'Sales',
    'Engg': 'Engineering',
    'Engineering': 'Engineering',
    'HR': 'Human Resources',
    'Human Resource': 'Human Resources',
    'Ops': 'Operations',
    'Operations': 'Operations',
    'Info Tech' : 'IT',
    'IT': 'IT',
    'Fin.': 'Finance',
    'Finance': 'Finance'
}

df['Department'] = (
    df['Department']
    .str.strip()
    .replace(department_mapping)
)

In [129]:
df['JoiningDate'] = pd.to_datetime(
    df['JoiningDate'],
    dayfirst=True,
    errors='coerce'
)

df['LastWorkingDate'] = pd.to_datetime(
    df['LastWorkingDate'],
    dayfirst=True,
    errors='coerce'
)

In [131]:
median_joining_date = df['JoiningDate'].median()
df['JoiningDate'] = df['JoiningDate'].fillna(median_joining_date)

<span style="font-family: Arial; font-size: 25px; font-weight: bold;">
Create Tenure Column for Active Empployees</span>

In [134]:
today = pd.Timestamp.today().normalize()

def calculate_tenure(row):
    if pd.isna(row['LastWorkingDate']):  # Active employee
        rd = relativedelta(today, row['JoiningDate'])
        return f"{rd.years} years {rd.months} months {rd.days} days"
    else:
        return None  # Not required for inactive employees

df['Active_Employee_Tenure'] = df.apply(calculate_tenure, axis=1)

In [136]:
df[['JoiningDate', 'LastWorkingDate', 'Active_Employee_Tenure']].head()

,JoiningDate,LastWorkingDate,Active_Employee_Tenure
0,2019-05-28,NaT,6 years 6 months 16 days
1,2020-06-08,NaT,5 years 6 months 6 days
2,2015-10-07,2016-10-18,None
3,2014-10-01,NaT,11 years 2 months 13 days
4,2020-08-10,NaT,5 years 4 months 4 days


In [138]:
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
Index: 47728 entries, 0 to 49999
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   EmployeeID              47728 non-null  int64         
 1   FullName                47728 non-null  object        
 2   FirstName               47728 non-null  object        
 3   LastName                47728 non-null  object        
 4   Age                     47728 non-null  int64         
 5   Gender                  47728 non-null  object        
 6   JobTitle                47728 non-null  object        
 7   Department              47728 non-null  object        
 8   Location                47728 non-null  object        
 9   EmploymentType          47728 non-null  object        
 10  Salary                  47728 non-null  float64       
 11  JoiningDate             47728 non-null  datetime64[ns]
 12  LastWorkingDate         11986 non-null  datetime64[

In [ ]:

df['JoiningDate'] = pd.to_datetime(df['JoiningDate'], dayfirst=True, errors='coerce').dt.strftime('%Y-%m-%d')
df['LastWorkingDate'] = pd.to_datetime(df['LastWorkingDate'], dayfirst=True, errors='coerce').dt.strftime('%Y-%m-%d')

In [97]:
# Export cleaned dataframe to CSV
df.to_csv(
    "HR_Payroll_data_v1.csv",
    index=False,
    encoding="utf-8"
)

In [99]:
print("Cleaned data successfully exported to HR_Payroll_data_v1")

Cleaned data successfully exported to HR_Payroll_data_v1
